# KNN per prevedere l'età delle lumache di mare
Per capire nella pratica come funziona questo algoritmo useremo il set di dati Abalone, questo set contiene misurazioni dell'età di un gran numero di abaloni. Solo per informazione, ecco come appare un abalone:

![abalone](dati/img/abalone.jpeg)

Come possiamo vedere, gli abaloni sono piccole lumache di mare che assomigliano un po' alle cozze. 

## La dichiarazione sul problema dell'abalone

L'età di un abalone può essere trovata tagliando il suo guscio e contando il numero di anelli sul guscio, in questo set di dati Abalone possiamo trovare le misurazioni dell'età di un gran numero di abalone insieme a molte altre misurazioni fisiche.

L'obiettivo del progetto è sviluppare un modello in grado di prevedere l'età di un abalone basandosi esclusivamente su altre misurazioni fisiche, ciò consentirebbe ai ricercatori di stimare l'età dell'abalone senza dover tagliare il guscio e contare gli anelli.

Applicheremo quindi un algoritmo kNN per trovare il punteggio di previsione più vicino possibile.

## Importazione del set di dati Abalone

Potremo scaricare manualmente il dataset e poi importarlo in Python utilizzando Pandas, ma è ancora più veloce lasciare fare tutto al codice:


In [ ]:
import pandas as pd
url = (
    "https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"
)
abalone = pd.read_csv(url, header=None)

abalone.head()

Possiamo vedere che mancano ancora i nomi delle colonne, possiamo trovare questi nomi nel repository di machine learning dell'UCI https://archive.ics.uci.edu/dataset/1/abalone e possiamo aggiungerli al tuo DataFrame come segue:

In [ ]:
abalone.columns = [
    "Sex",
    "Length",
    "Diameter",
    "Height",
    "Whole weight",
    "Shucked weight",
    "Viscera weight",
    "Shell weight",
    "Rings",
]

I dati importati ora dovrebbero essere più comprensibili, ma c'è un'altra cosa che dovremo fare: rimuovere la colonna Sex. L'obiettivo dell'esercizio attuale è utilizzare misurazioni fisiche per prevedere l'età dell'abalone, poiché il sesso non è una misura puramente fisica, lo rimuoveremo dal set di dati utilizzando .drop:

In [ ]:
abalone = abalone.drop("Sex", axis=1)

## Statistiche descrittive dal set di dati Abalone
Quando lavoriamo sull'apprendimento automatico, dobbiamo avere un'idea dei dati con cui stiamo lavorando.

La variabile target di questo esercizio è Rings, quindi possiamo iniziare con quella, un istogramma ci fornirà una panoramica rapida e utile delle fasce di età che possiamo aspettarci:

In [ ]:
import matplotlib.pyplot as plt
abalone["Rings"].hist()
plt.show()

L'istogramma mostra che la maggior parte degli abaloni nel set di dati ha tra i cinque e i quindici anelli, ma che è possibile arrivare fino a venticinque anelli, quindi capiamo subito che gli abaloni più vecchi sono sottorappresentati in questo set di dati, ciò sembra intuitivo, poiché le distribuzioni per età sono generalmente distorte in questo modo a causa dei processi naturali.

Una seconda esplorazione rilevante è scoprire quali variabili, se presenti, hanno una forte correlazione con l’età, una forte correlazione tra una variabile indipendente e la variabile target sarebbe un buon segno, poiché confermerebbe che le misurazioni fisiche e l'età sono correlate.

Come sappiamo, è possibile osservare la matrice di correlazione completa con correlation_matrix , le correlazioni più importanti sono quelle con la variabile target Rings. 
Possiamo ottenere quelle correlazioni in questo modo:

In [ ]:
correlation_matrix = abalone.corr()
correlation_matrix["Rings"]

Ora guardando i coefficienti di correlazione Rings con le altre variabili sappiamo che più si avvicinano a 1, maggiore è la correlazione.

Si può concludere che esiste almeno una certa correlazione tra le misurazioni fisiche degli abaloni adulti e la loro età, ma non è nemmeno molto alta, correlazioni molto elevate significano che possiamo aspettarci un processo di modellazione semplice, in questo caso invece dovremo provare a vedere quali risultati possiamo ottenere utilizzando l'algoritmo kNN.

## Definire il "più vicino" utilizzando una definizione matematica di distanza

Abbiamo detto che per trovare i punti dati più vicini al punto che dobbiamo prevedere, possiamo utilizzare una definizione matematica di distanza chiamata distanza euclidea. 

Senza tornare nello specifico sulla formula, per applicarla ai nostri dati, dobbiamo capire che i punti sono in realtà vettori, quindi è possibile calcolare la distanza tra loro calcolando la norma del vettore differenza.

In Python possiamo farlo usando linalg.norm() da NumPy. 
Ecco un esempio:


In [ ]:
import numpy as np
a = np.array([2, 2])
b = np.array([4, 4])
np.linalg.norm(a - b)

In questo blocco di codice definiamo i punti dati come vettori, quindi calcoliamo la differenza tra due punti dati con norm(). In questo modo si ottiene direttamente la distanza tra due punti multidimensionali, anche se i punti sono multidimensionali, la distanza tra loro è ancora uno scalare o un singolo valore.

## Trovare i k vicini

Ora che conosciamo un modo per calcolare la distanza da qualsiasi punto a qualsiasi punto, possiamo usarlo per trovare i più vicini a un punto su cui vogliamo fare una previsione.

Dobbiamo trovare un numero di vicini e quel numero è dato da k, il valore minimo di k è 1, ciò significa utilizzare un solo vicino per la previsione. Il massimo di vicini è invece il numero di punti dati di cui disponi, ciò significa utilizzare tutti i vicini. Come sappiamo quindi, il valore di k è qualcosa che l'utente definisce e in questo ci aiutano gli strumenti di ottimizzazione.

Ora, per trovare i vicini più vicini in NumPy, torniamo al set di dati Abalone, come visto, dobbiamo definire le distanze sui vettori delle variabili indipendenti, quindi dobbiamo prima inserire il DataFrame pandas in un array NumPy usando l'attributo .values:

In [ ]:
X = abalone.drop("Rings", axis=1)
X = X.values
y = abalone["Rings"]
y = y.values

Ora possiamo provare ad applicare l'algoritmo kNN con k= 3 su un nuovo abalone che abbia le seguenti misure fisiche:



In [ ]:
new_data_point = np.array([
    0.569552,
    0.446407,
    0.154437,
    1.016849,
    0.439051,
    0.222526,
    0.291208,
])

Il passaggio successivo consiste nel calcolare le distanze tra questo nuovo punto e ciascuno dei punti nel set di dati Abalone utilizzando il seguente codice:

In [ ]:
distances = np.linalg.norm(X - new_data_point, axis=1)

Ora abbiamo un vettore di distanze e dobbiamo scoprire quali sono i tre punti più vicini, per fare ciò è necessario trovare gli ID delle distanze minime. Possiamo utilizzare un metodo chiamato .argsort() per ordinare l'array dal più basso al più alto e possiamo prendere i primi k elementi per ottenere gli indici dei k più vicini:

In [ ]:
k = 3
nearest_neighbor_ids = distances.argsort()[:k]
nearest_neighbor_ids

## Voto o media di più vicini

Dopo aver identificato gli indici dei tre più vicini dal nostro nuovo abalone di età sconosciuta, dobbiamo combinare questi vicini in una previsione.

Come primo passo, dobbiamo trovare le età per questi tre vicini:

In [ ]:
nearest_neighbor_rings = y[nearest_neighbor_ids]
nearest_neighbor_rings

## Media per la regressione
Come sappiamo nei problemi di regressione, la variabile target è numerica, combiniamo quindi i più vicini in un'unica previsione prendendo la media dei loro valori della variabile target:

In [ ]:
prediction = nearest_neighbor_rings.mean()

Otterremo un valore di 10 for prediction. Ciò significa che la previsione dei 3 vicini più vicini per il nuovo Abalone è 10.

## Modalità di classificazione
Nei problemi di classificazione invece la variabile obiettivo è categorica, quindi non usiamo la media ma la moda, che come già sappiamo, è il valore che ricorre più spesso, ciò significa che contiamo le classi di tutti i vicini e mantieniamo la classe più comune, la previsione è quindi il valore che ricorre più spesso tra i vicini.

Se esistono più mode, ci sono più soluzioni possibili, potremmo selezionare un vincitore finale in modo casuale tra i vincitori, potremmo anche prendere la decisione finale in base alle distanze dei vicini, nel qual caso verrebbe mantenuta la moda dei più vicini.

Poiché l'esempio dell'abalone non è un caso di classificazione, partiremo da altri dati di esempio.

Come abbiamo già visto, è possibile quindi calcolare la moda utilizzando la funzione SciPy mode(), ma dalla versione 1.11.0 è stata tolta la possibilità di calcolare la moda su dati non numerici, creremo quindi una funzione personalizzata:

In [ ]:
class_neighbors = np.array(["A", "B", "B", "C"])


def mode(lst):
     
    # creating a dictionary
    freq = {}
    for i in lst:
       
        # mapping each value of list to a 
        # dictionary
        freq.setdefault(i, 0)
        freq[i] += 1
         
    # finding maximum value of dictionary
    hf = max(freq.values())
     
    # creating an empty list
    hflst = []
     
    # using for loop we are checking for most 
    # repeated value
    for i, j in freq.items():
        if j == hf:
            hflst.append(i)
             
    # returning the result
    return hflst

# calling mode() function and passing list
# as argument
print(mode(class_neighbors))

## Implementiamo l'algoritmo kNN in Python utilizzando scikit-learn
Sebbene codificare un algoritmo da zero è ottimo per scopi di apprendimento, di solito non è molto pratico quando si lavora su un'attività di machine learning, vedremo ora quindi l'implementazione dell'algoritmo kNN utilizzato scikit-learn.

### Suddivisione dei dati in set di training e test per la valutazione del modello
In questa sezione valuteremo la qualità del modello kNN di abalone, nelle sezioni precedenti avevamo un focus tecnico, ma ora avremo un punto di vista più pragmatico e orientato ai risultati.

Esistono diversi modi per valutare i modelli, ma come abbiamo già visto il più comune e semplice è la suddivisione in train-test:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=12345
)

### Adattamento di una regressione kNN in scikit-learn al set di dati Abalone
Come sappiamo, per adattare il modello da scikit-learn iniziamo creando un modello della classe corretta dopo scegliamo i valori per i nostri iperparametri.
Per l'algoritmo kNN, dovremo scegliere il valore per k, che viene chiamato n_neighbors:

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=3)

Dopo aver scelto i tre più vicini, facciamo allenare il modello con .fit()

In [ ]:
knn_model.fit(X_train, y_train)

### Utilizziamo scikit-learn per ispezionare l'adattamento del modello
Esaminiamo ora alcune funzioni che è possibile utilizzare per valutare l'adattamento.

Come sappiamo sono disponibili molti parametri di valutazione per la regressione, ma utilizzeremo ora uno dei più comuni, l' errore quadratico medio (RMSE) :



In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
train_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

In questo codice, abbiamo calcolato l'RMSE sui dati di addestramento, ma per un risultato più realistico, dovremo valutare le prestazioni sui dati non inclusi nel modello. Questo è il motivo per cui per cui abbiamo mantenuto il set di test separato. Andiamo quindi a utilizzare la stessa funzione di prima sui dati di test:

In [ ]:
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
rmse

Questo RMSE più realistico è leggermente più alto di prima, possiamo interpretarlo come se avesse, in media, un errore di 1.65 anni con i dati di train e 2.37 anni con i dati di test.

Fino ad ora, abbiamo utilizzato solo l'algoritmo kNN scikit-learn pronto all'uso, non abbiamo ancora effettuato alcuna regolazione degli iperparametri e una scelta casuale per k. 
È possibile osservare una differenza relativamente ampia tra l'RMSE sui dati di addestramento e l'RMSE sui dati di test, ciò significa che il modello soffre di un adattamento eccessivo ai dati di addestramento: non si generalizza bene.

### Tracciare l'adattamento del tuo modello
Un'ultima cosa da considerare prima di iniziare a migliorare il modello è l'effettiva vestibilità del tuo modello, per capire cosa ha imparato il modello, possiamo visualizzare come sono state effettuate le previsioni utilizzando Matplotlib:

In [ ]:
import seaborn as sns
cmap = sns.cubehelix_palette(as_cmap=True)
f, ax = plt.subplots()
points = ax.scatter(
    X_test[:, 0], X_test[:, 1], c=test_preds, s=50, cmap=cmap
)
f.colorbar(points)
plt.show()

In questo blocco di codice, abbiamo utilizzato Seaborn per creare un grafico a dispersione della prima e della seconda colonna di X_test con le prime due colonne di array X_test[:,0]e X_test[:,1]. Ricordiamo che le prime due colonne sono Length e Diameter che Sono fortemente correlate, come abbiamo visto nella tabella delle correlazioni.

Utilizziamo poi c per specificare che i valori previsti ( test_preds) devono essere utilizzati come barra dei colori. L'argomento s invece viene utilizzato per specificare la dimensione dei punti nel grafico a dispersione. Infine cmapper si utilizza per specificare la mappa dei colori.

#### Nel grafico che otteniamo
ogni punto è un abalone del set di prova, con la sua lunghezza effettiva e il suo diametro effettivo rispettivamente sugli assi X e Y. Il colore del punto riflette l'età prevista, possiamo vedere che più un abalone è lungo e grande, maggiore è la sua età prevista. Questo è logico ed è un segnale positivo, significa che il modello sta imparando qualcosa che sembra corretto.

Per confermare se questa tendenza esiste nei dati effettivi dell'abalone, possiamo fare lo stesso per i valori effettivi semplicemente sostituendo la variabile utilizzata per c:

In [ ]:
cmap = sns.cubehelix_palette(as_cmap=True)
f, ax = plt.subplots()
points = ax.scatter(
    X_test[:, 0], X_test[:, 1], c=y_test, s=50, cmap=cmap
)
f.colorbar(points)
plt.show()

Il grafico conferma che la tendenza che il modello sta apprendendo ha davvero senso.

### Migliorare le prestazioni di kNN nell'uso di scikit-learnGridSearchCV

In questo esempio abbiamo sempre lavorato con k=3, ma come sappiamo il valore migliore k è qualcosa che dobbiamo trovare empiricamente per ciascun set di dati.

Per trovare il miglior valore per k, utilizzeremo come già visto in precedenza lo strumento chiamato GridSearchCV:

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {"n_neighbors": range(1, 50)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train)

Come abbiamo già visto, GridSearchCV adatta ripetutamente i regressori kNN su una parte dei dati e testa le prestazioni sulla restante parte dei dati, facendo ciò  si otterrà una stima affidabile delle prestazioni predittive di ciascuno dei valori per k (In questo esempio vengono testati i valori da 1 a 50).

Alla fine potremo accedere al valore più perfomanete con .best_params_:

In [ ]:
gridsearch.best_params_

Possiamo quindi vedere che la scelta 25 come valore di k produrrà le migliori prestazioni predittive. Ora che sappiamo quale k è il miglior valore, possiamo vedere come influisce sulle prestazioni dei dati di train e test:

In [ ]:
train_preds_grid = gridsearch.predict(X_train)
train_mse = mean_squared_error(y_train, train_preds_grid)
train_rmse = sqrt(train_mse)
test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = sqrt(test_mse)

print(train_rmse)

print(test_rmse)

Possiamo vedere che l'errore di addestramento è peggiore di prima, ma l'errore di test è migliore di prima, ciò significa che grazie al nostro lavoro il modello si adatta meno fedelmente ai dati di addestramento, quindi l'utilizzo GridSearchCV per trovare un valore k ha ridotto il problema dell'adattamento eccessivo ai dati di training.

### Aggiunta della media ponderata dei vicini in base alla distanza

Vedremo ora come migliorare ancora di più le prestazioni, testeremo quindi se le prestazioni del modello saranno migliori quando eseguiamo la previsione utilizzando una media ponderata anziché una media regolare, ciò significa che i vicini più lontani influenzeranno meno fortemente la previsione.

Possiamo farlo impostando l'iperparametro weights sul valore di "distance", questo cambiamento potrebbe avere un impatto sul valore ottimale di k, pertanto, riutilizzeremo nuovamente GridSearchCV per capire quale tipo di media dovremmo utilizzare:

In [ ]:
parameters = {
    "n_neighbors": range(1, 50),
    "weights": ["uniform", "distance"],
}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train)



gridsearch.best_params_

test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = sqrt(test_mse)
test_rmse

L'applicazione di una media ponderata anziché di una media regolare ha ridotto l'errore di previsione da 2.17 a 2.1634, anche se questo non è un enorme miglioramento, è comunque un miglioramento e sopratutto potrebbe essere invece un miglioramento molto più evidente su altri set di dati.

### Ulteriore miglioramento di kNN in scikit-learn con Bagging

Come terzo passaggio per l'ottimizzazione dell'algoritmo kNN, è possibile utilizzare il bagging un metodo d'insieme, ovvero un metodo che prende un modello di apprendimento automatico relativamente semplice e si adatta a un gran numero di tali modelli con lievi variazioni in ogni adattamento. Il bagging utilizza spesso alberi decisionali (che vedremo più avanti), ma anche con l'algoritmo kNN funziona perfettamente.

I metodi insieme sono spesso più performanti dei modelli singoli, un modello può sbagliarsi di tanto in tanto, ma la media di un centinaio di modelli dovrebbe sbagliarsi meno spesso. È probabile che gli errori dei diversi modelli individuali si medino a vicenda e la previsione risultante sarà meno variabile.

Possiamo utilizzare scikit-learn per applicare il bagging alla regressione kNN utilizzando i seguenti passaggi:

1. Per prima cosa, creiamo il file KNeighborsRegressor con le migliori scelte per k e weights che abbiamo ottenuto da GridSearchCV:

In [ ]:
best_k = gridsearch.best_params_["n_neighbors"]
best_weights = gridsearch.best_params_["weights"]
bagged_knn = KNeighborsRegressor(
    n_neighbors=best_k, weights=best_weights
)

2. Quindi importiamo la classe BaggingRegressor da scikit-learn e creiamo una nuova istanza con 100 stimatori utilizzando il modello bagged_knn:


In [ ]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(bagged_knn, n_estimators=100)

3. Ora fittiamo i dati e poi possiamo fare una previsione e calcolare l'RMSE per vedere se è migliorato:

In [ ]:
bagging_model.fit(X_train, y_train)
test_preds_grid = bagging_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = sqrt(test_mse)
test_rmse

L'errore di previsione sul kNN è 2.1616, che è leggermente inferiore all'errore precedente ottenuto. 
### Confronto dei quattro modelli
In tre passaggi incrementali abbiamo migliorato ulteriromente le prestazioni predittive dell'algoritmo. 

Di seguito la tabella che mostra un riepilogo dei diversi modelli e delle loro prestazioni:

<table class="table table-hover">
<thead>
<tr>
<th><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Modello</font></font></th>
<th><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Errore</font></font></th>
</tr>
</thead>
<tbody>
<tr>
<td><font style="vertical-align: inherit;"><font style="vertical-align: inherit;"><code>k</code> Arbitrario</font></font></td>
<td><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2.37</font></font></td>
</tr>
<tr>
<td><code>GridSearchCV</code> <font style="vertical-align: inherit;"><font style="vertical-align: inherit;"> per</font></font> <code>k</code></td>
<td><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2.17</font></font></td>
</tr>
<tr>
<td><code>GridSearchCV</code> <font style="vertical-align: inherit;"><font style="vertical-align: inherit;"> per </font></font> <code>k</code> <font style="vertical-align: inherit;"><font style="vertical-align: inherit;"> e </font></font> <code>weights</code></td>
<td><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2.1634</font></font></td>
</tr>
<tr>
<td><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bagging e </font></font> <code>GridSearchCV</code></td>
<td><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2.1616</font></font></td>
</tr>
</tbody>
</table>

Vediamo i quattro modelli dal più semplice al più complesso. L'ordine di complessità corrisponde all'ordine delle metriche di errore. Il modello con un random k ha ottenuto i risultati peggiori, mentre il modello con il bagging e GridSearchCV i risultati migliori.

## Esercizio 1
Utilizzare l'algoritmo k-Nearest Neighbors (k-NN) per la classificazione utilizzando il dataset "Iris" disponibile nella libreria scikit-learn:

### Passaggi da completare:

- Caricamento del dataset:

1. Carica il dataset "Iris" dalla libreria scikit-learn.
2. Esplora il dataset per comprendere le caratteristiche presenti, i loro tipi e la distribuzione delle classi di output.

- Preprocessing dei dati:

1. Dividi il dataset in features (variabili indipendenti) e target (variabile dipendente).
2. Dividi il dataset in training set e test set utilizzando una proporzione del 80-20.

- Creazione del modello k-NN:

1. Crea un modello k-NN utilizzando il numero di vicini desiderato.

- Addestramento del modello:

1. Addestra il modello k-NN sul training set.

- Valutazione del modello:

1. Valuta le prestazioni del modello utilizzando il test set.
2. Calcola l'accuratezza del modello.
3. Visualizza il report di classificazione che include precision, recall e F1-score per ogni classe.
4. Visualizza la matrice di confusione per valutare le prestazioni del modello.

## Esercizio 2
In questo esercizio, userete il dataset "Breast Cancer Wisconsin (Diagnostic)" disponibile nella libreria scikit-learn.

### Passaggi da completare:

- Caricamento del dataset:

1. Carica il dataset "Breast Cancer Wisconsin (Diagnostic)" dalla libreria scikit-learn.
2. Esplora il dataset per comprendere le caratteristiche presenti, i loro tipi e la distribuzione delle classi di output.

- Preprocessing dei dati:

1. Dividi il dataset in features (variabili indipendenti) e target (variabile dipendente).
2. Dividi il dataset in training set e test set utilizzando una proporzione del 70-30.

- Standardizzazione dei dati:

1. Standardizza le features utilizzando lo StandardScaler di scikit-learn.

- Creazione del modello k-NN:

1. Crea un modello k-NN specificando il numero di vicini desiderato.

- Addestramento del modello:

1. Addestra il modello k-NN sul training set.

- Valutazione del modello:

1. Valuta le prestazioni del modello utilizzando il test set.
2. Calcola l'accuratezza del modello.
3. Visualizza il report di classificazione che include precision, recall e F1-score per ogni classe.
4. Visualizza la matrice di confusione per valutare le prestazioni del modello.